# **Data Collection**

## Objectives

* Retrieve data from the Kaggle image dataset provided by Farmy & Foods and prepare it for subsequent processing.

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Dataset Generation: https://www.kaggle.com/codeinstitute/cherry-leaves

### Additional Comments

* The client has provided the data under a non-disclosure agreement (NDA), necessitating its restricted sharing solely with project-involved professionals.
Regarding the dataset's nature, it implies binary image classification, distinguishing between healthy cherry leaves and those infected with powdery mildew.


---

## Import Packages

In [1]:
%pip install -r /workspace/mildew-detection-in-cherry-leaves/requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy
import os

## Change Working directory 

* Change working directory to current one

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves/jupyter_notebooks'

In [4]:
import os
current_dir = os.getcwd()
current_dir
os.chdir('/workspace/mildew-detection-in-cherry-leaves')
print("You set a new current directory")

You set a new current directory


* Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves'

## Install Kaggle 

In [6]:
%pip install kaggle==1.5.12


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

* Modify the Kaggle configuration directory and adjust the permissions of the Kaggle authentication JSON file.

In [7]:
os.environ['/workspace/mildew-detection-in-cherry-leaves/jupyter_notebooks'] = os.getcwd()
! chmod 600 kaggle.json

* Get the dataset path from the [Kaggle url](https://www.kaggle.com/codeinstitute/cherry-leaves) and set your destination folder.

In [8]:
KaggleDatasetPath = "/codeinstitute/cherry-leaves_dataset"
DestinationFolder = "inputs/cherry-leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Traceback (most recent call last):
  File "/home/gitpod/.pyenv/versions/3.8.18/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/gitpod/.pyenv/versions/3.8.18/lib/python3.8/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/gitpod/.pyenv/versions/3.8.18/lib/python3.8/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/gitpod/.kaggle. Or use the environment method.


---

* Unzip the downloaded file, delete the zip file

In [17]:
import zipfile
with zipfile.ZipFile('inputs/cherry-leaves_dataset/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('inputs/cherry-leaves_dataset')


In [10]:
!ls

inputs		   kaggle.json	Procfile   requirements.txt  setup.sh
jupyter_notebooks  outputs	README.md  runtime.txt


## Data Preparation 

* Checking and removing non image files

In [19]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir) 
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        
            #print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))
    
    

In [20]:
remove_non_image_file(my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves/')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


# Split train validation test set

* We are dividing the data set into 3 subsets by using the "split_train_validation_test_images" function: <br/>
    * Training Set<br/> 
    * Test Set  <br/>
    * Validation Set 

In [26]:
import os
import shutil
import random
import joblib

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  # gets classes labels
  labels = os.listdir(my_data_dir) # it should get only the folder name
  if 'test' in labels:
    pass
  else: 
    # create train, test folders with classess labels sub-folder
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          # move given file to train set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          # move given file to validation set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          # move given file to test set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)
    
    # Print the number of files in each set after splitting
    print("Number of files in Train set:")
    for label in labels:
        train_files = os.listdir(my_data_dir + '/train/' + label)
        print(f"Class {label}: {len(train_files)}")

    print("\nNumber of files in Validation set:")
    for label in labels:
        validation_files = os.listdir(my_data_dir + '/validation/' + label)
        print(f"Class {label}: {len(validation_files)}")

    print("\nNumber of files in Test set:")
    for label in labels:
        test_files = os.listdir(my_data_dir + '/test/' + label)
        print(f"Class {label}: {len(test_files)}")

* By convention:

    * The training set comprises 70% of the data.
    * The validation set comprises 10% of the data.
    * The test set comprises 20% of the data.

In [27]:
split_train_validation_test_images(my_data_dir = f"inputs/cherry-leaves_dataset/cherry-leaves",
                        train_set_ratio = 0.7,
                        validation_set_ratio=0.1,
                        test_set_ratio=0.2
                        )


* Check number of images on each, Train, Validation and Test Sets.

In [31]:
test_dir_healthy = 'inputs/cherry-leaves_dataset/cherry-leaves/test/healthy/'
test_dir_powdery = 'inputs/cherry-leaves_dataset/cherry-leaves/test/powdery_mildew/'

train_dir_healthy= 'inputs/cherry-leaves_dataset/cherry-leaves/train/healthy/'
train_dir_powdery = 'inputs/cherry-leaves_dataset/cherry-leaves/train/powdery_mildew/'
validation_dir_healthy = 'inputs/cherry-leaves_dataset/cherry-leaves/validation/healthy/'
validation_dir_powdery = 'inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/'

# Function to count the number of files in a directory
def count_files(my_data_dir):
    return len(os.listdir(my_data_dir))

# Count the number of images in each directory
test_count = count_files(test_dir_healthy)
test_count = count_files(test_dir_powdery)
train_count = count_files(train_dir_healthy)
train_count = count_files(train_dir_powdery)
validation_count = count_files(validation_dir_healthy)
validation_count = count_files(validation_dir_powdery)

# Print the counts
print("Number of images in the test set:", test_count)
print("Number of images in the train set:", train_count)
print("Number of images in the validation set:", validation_count)

Number of images in the test set: 422
Number of images in the train set: 1472
Number of images in the validation set: 210


## Push files to Repo 

In [33]:
import os
try:
 # Change directory to your repository
    os.chdir("/workspace/mildew-detection-in-cherry-leaves/")

    # Add all files to the staging area
    os.system("git add .")

    # Commit the changes with a message
    commit_message = "Save the data Collection notebook "
    os.system(f"git commit -m '{commit_message}'")

    # Push changes to the origin (assuming you have already set up your origin)
    os.system("git push origin main")  # Replace 'main' with your branch name if it's different
except Exception as e:
    print(e)

[main 4f70ff4] Save the data Collection notebook
 1931 files changed, 212 insertions(+), 94 deletions(-)
 create mode 100644 inputs/cherry-leaves_dataset/archive.zip
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/0008f3d3-2f85-4973-be9a-1b520b8b59fc___JR_HL 4092.JPG (100%)
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/0086a8c7-1440-423e-939d-d0567b1fc4e4___JR_HL 4277_flipTB.JPG (100%)
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/01958ee7-f585-4956-90aa-a40dc79102d4___JR_HL 9836_180deg.JPG (100%)
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/02818f40-58e6-4c7d-bfbc-372d89b2045b___JR_HL 4259_180deg.JPG (100%)
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/02818f40-58e6-4c7d-bfbc-372d89b2045b___JR_HL 4259_flipTB.JPG (100%)
 rename inputs/cherry-leaves_dataset/cherry-leaves/{train => test}/healthy/04189592-5db7-441e-a831-eaaa2d7f5d21___JR_HL 9867_f

remote: warning: See https://gh.io/lfs for more information.        
remote: warning: File inputs/cherry-leaves_dataset/archive.zip is 55.02 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
To https://github.com/FerchaPombo/mildew-detection-in-cherry-leaves.git
   25f039e..4f70ff4  main -> main


---

## Conclusions and Next Steps

* In conclusion, the dataset acquired from FarmFood Inc. comprises two folders containing images of healthy and diseased leaves. No missing data was identified, and all data is in image format (jpg, jpeg, png). No additional data cleaning procedures were deemed necessary.

* The dataset has been effectively partitioned into three sets—Train, Validation, and Test sets—following a conventional split ratio.

* Subsequent stages of the data cycle will involve the implementation of data visualization techniques to provide a comprehensive graphical depiction for better understanding of the dataset.